In [1]:
# This script extracts the title, link, and short description of search results on Google 

#import all libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import time, sys, requests, random


# import libs, authorize gspread  
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-eaf0e0e671cb.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)


In [2]:
# Global variables

# paths/baseurls
spreadsheet_id = '1vFXonFCyUlEKa1f0s5tvHCKeTek_sAv7rUPYfYss0Qo'
companyFilePath = '/Users/zacharywong/Downloads/DigitalHealthWebscrape - Companies.csv'
driver_path = '/Users/zacharywong/Documents/Work/Portfolio/DigitalHealthWebscrape/chromedriver'
pathtoFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Deliverables/'
googleurl = 'https://www.google.com/'

#ints 
adjustDenominator = 3
maxbackOff = 120
maxResult = 1
waitTime = 5
waitRun = 0.3

#bools
useURL = False
useName = False


In [3]:
def wait():
    pass
    global waitRun
    sleepTime = waitRun + random.uniform(0, 1)
    #print('sleepTime: ', sleepTime)
    time.sleep(sleepTime)
    waitRun = waitRun*2
    #print('waitRun: ', waitRun)
    if (waitRun >= maxbackOff):
        sys.exit("error: read from sheets quota exceeded")

In [4]:
# Helper function: reads in values from DigitalHealthWebscrape google sheet
# Need spreadsheet ID and the cell address where the value should be read in 
# returns the value 

def readinValue(cellLocation, sheetIndex):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndex)
    try:
        value = worksheet.acell(cellLocation).value
    except:
        wait()
    return value

In [5]:
def readWebsites():
    links = []
    names = []
    df = pd.read_csv(companyFilePath, usecols=range(1,3))
    df['Name']=df['Name'].fillna(' ')
    df['Link']=df['Link'].fillna(' ')
    names = df['Name'].tolist()
    links = df['Link'].tolist()
    #print(df)
    return links, names 
    

In [6]:
def readKeyWords():
    keywords = []
    row = 2;
    sheetIndex = 1
    isDone = False
    while (isDone == False):
        cellLocationKeyWords = 'C' + str(row)
        try:
            keyword = readinValue(cellLocationKeyWords, sheetIndex)
            keywords.append(keyword)
            if(keyword == None):
                isDone = True
                break
            else:
                row += 1 
        except:
            wait()
    keywords = keywords[0:len(keywords)-1]
    return keywords

In [7]:
def readCalculateLiklihood():
    #print(links, names)
    sheetIndex = 1
    cellLocationLiklihood = 'B2'
    try:
        calculateLiklihoodInput = readinValue(cellLocationLiklihood, sheetIndex)
        if (calculateLiklihoodInput == 'Yes'):
            calculateLiklihood = True
        else:
            calculateLiklihood = False
    except:
        wait()
    return calculateLiklihood

In [8]:
def readInput():
    links, names = readWebsites()
    calculateLiklihood = readCalculateLiklihood()
    keywords = readKeyWords()
    return links, names, calculateLiklihood, keywords

In [9]:
# helper function to extract links from 1 page of results
def extractLinks(soup):
    links = []
    linksTag = 'div'
    linksClass = 'yuRUbf'
    linksAttr = 'href'
    searchLinks = soup.find_all(linksTag, class_ = linksClass)
    for h in searchLinks:
        link = h.a.get(linksAttr)
        links.append(link)
    return links 

In [10]:
def extractTitles(soup):
    titles = [] 
    titlesClass = 'LC20lb MBeuO DKV0Md'
    titlesTag = 'h3'
    searchTitles = soup.find_all(titlesTag, class_= titlesClass)
    for h in searchTitles:
        titles.append(h.text)
    return titles

In [11]:
# helper function to extract texts from 1 page of results
def extractTexts(soup):
    texts = []
    textsClass = 'VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf'
    textsTag = 'div'
    searchText = soup.find_all(textsTag, class_= textsClass)
    for h in searchText:
        fullText = h.text
        try:
            splitText = fullText.split('— ', 1)
            text = splitText[1]
            texts.append(text);
        except:
            texts.append(fullText)
    return texts;

In [12]:
def calculateLiklihoods(links, keywords):
    #keywords = ['personalized', 'personalization', 'machine-learning', 'AI', 'Artificial Intelligence', '24/7', 'democratizing']
    liklihoods = []
    detectedWordsAll = []
    liklihoodDenom = len(keywords) - adjustDenominator
    for url in links: 
        detectedWords = []
        output = ''
        count = 0
        liklihood = 0
        ignore = ['[document]', 'a', 'article', 'label', 'script', 'style']


        res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        html_page = res.content
        soup = bs(html_page, 'html.parser')
        #print("Encoded method :" + url + ": ", soup.original_encoding)
        text = soup.find_all(text=True)
       
        for t in text:
            if t.parent.name not in ignore:
                output += '{} '.format(t)

        # analyze the webpage to detect keywords 
        outputSub = output.split(' ')
        #print(outputSub)
        for word in keywords: 
            if (word in outputSub or word.capitalize() in outputSub or word + '\n' in outputSub):
                #print('"' + word +'"' + ' detected')
                detectedWords.append(word)
                count +=1 
            #else:
                #print(word + ' is not there')
        liklihood = round((count / liklihoodDenom), 2)
        liklihoods.append(liklihood)
        detectedWordsAll.append(detectedWords)
        #detectedWordsAll = ', '.join(str(keyword) for keyword in detectedWordsAll)
    #print("detectedWordsAll", detectedWordsAll)
    return liklihoods, detectedWordsAll

In [13]:
# set the keyword you want to search for depending on whether link or name is given
# we find the search bar using its name attribute value (q)

def searchGoogle(index, linksInput, namesInput, googleurl, useURL, waitTime, driver):
    #print("index: ", index)
    #print("Number of Companies: " + str((len(linksInput))))
    #print("links: ", linksInput)
    #print("names: ", namesInput)
    siteURL = linksInput[index]
    #print("siteURL: ", siteURL)
    websiteName = namesInput[index]
    #print("websiteName: ", websiteName)
    driver.get(googleurl)
    searchBar = driver.find_element(By.NAME, 'q')
    
    # Booleans
    if (siteURL == ' '):
        useURL = False; 
        useName = True; 
    else:
        useURL = True; 
        useName = False; 

    # first we send our keyword to the search bar followed by the enter # key depending on using URL or website name 

    if (useURL):
        query = "site: " + siteURL
        print('query: ' + query)
        try:
            searchBar.send_keys(query)
            searchBar.send_keys('\n')
        except Exception as e : 
            WebDriverWait(driver, waitTime).until(EC.presence_of_element_located((By.NAME, 'q')))
            searchBar.send_keys(query)
            searchBar.send_keys('\n')
    else:
        query = websiteName
        print('query: ' + query)
        searchBar.send_keys(query)
        searchBar.send_keys('\n')

In [14]:
#capture links, header, and text
#pageInfo is a list of dictionaries for each page with keys/value pairs: header, link, text
# extract and load each page of results to pageInfo 
def parseHTML(driver, calculateLiklihood, keywords, maxResult, pageInfo):
    soup = bs(driver.page_source, 'html.parser')
    links = extractLinks(soup);
    texts = extractTexts(soup);
    titles = extractTitles(soup);
    if (calculateLiklihood):
        liklihoods, detectedWordsAll = calculateLiklihoods(links[0:1], keywords)
        #print('detectedWordsAllStringForm: ' + str(detectedWordsAll))
        #detectedWordsAll = ', '.join([str(keyword) for keyword in detectedWordsAll])
        #print("detectedWordsAll String Form: " + detectedWordsAll)
        pageInfo = addToPageInfo(titles, links, texts, maxResult, pageInfo, liklihoods, detectedWordsAll)
    else:
        pageInfo = addToPageInfo(titles, links, texts, maxResult, pageInfo, liklihoods = None, detectedWordsAll = None)

    return pageInfo

In [15]:
# helper function to add 1 page of results to pageInfo list
def addToPageInfo(titles, links, texts, maxResult, pageInfo, liklihoods, detectedWordsAll):
    index = 0;
    while (index < maxResult):
        #print("texts: ", texts)
        #print("liklihoods: ", liklihoods)
        #print("length of links list: " + str((len(linksInput)-1)))
        #print("links: ", links)
        #print("titles: ", titles)
        #print('detectedWordsAll', detectedWordsAll)
        #create new dictionary of each search results' title, link, and text
        if (liklihoods == None):
            pageInfo.append({"Title": titles[index], "Link": links[index], "About": texts[index]})
        else:
            #print("detectedWordsString: ", detectedWordsAll)
            detectedWords = str(detectedWordsAll[index])
            pageInfo.append({"Probability": liklihoods[index], "Title": titles[index], "Link": links[index], "About": texts[index],  "Detected Words": detectedWords})
        index += 1
    return pageInfo


In [16]:
def exportCSV(df, pathtoFile):
    # convert pageInfo to pandas dataframe and export as csv 
    df.to_csv(pathtoFile + 'WebScrapeDeliverable.csv')

In [17]:
def updateSpreadSheet(df):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(2)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    

In [18]:
def exportDeliverables(pageInfo, pathtoFile):
    df = pd.DataFrame(pageInfo)
    df = df.sort_values(by = ['Probability'], ascending=False)

    #print(df.columns.values.tolist())
    #print(df.values.tolist())
    #print(pageInfo)
    exportCSV(df, pathtoFile)
    updateSpreadSheet(df)
    return df 

In [19]:
def run():
    index = 0
    linksInput, namesInput, calculateLiklihood, keywords = readInput()
    print("Number of Companies: " + str((len(linksInput))))
    # Access chromedriver and determine path 
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    # list of dictionaries with key/value pairs: title, link, text
    # Contains all information for all search results 
    pageInfo = []

    while (index < len(linksInput)):
        searchGoogle(index, linksInput, namesInput, googleurl, useURL, waitTime, driver)
        for page in range(0, 1):
            pageInfo = parseHTML(driver, calculateLiklihood, keywords, maxResult, pageInfo)
    #print(pageInfo)
            index += 1
        time.sleep(waitRun)
    df = exportDeliverables(pageInfo, pathtoFile)
    return df



In [20]:
# Run the script 
# implement exponential backoff algorithm to prevent exceeding read quota from sheets
startTime = time.time()
df = run()
endTime = time.time()
timeElapsed = endTime - startTime
print('Time Elapsed: ', timeElapsed)
df.head()


Number of Companies: 11
query: OneRemission
query: Youper
query: Babylon Health
query: Florence Health
query: Healthily
query: Ada Health
query: Sensely
query: Buoy Health
query: Infermedica
query: GYANT
query: Woebot
Time Elapsed:  48.9050931930542


,Probability,Title,Link,About,Detected Words
2,0.6,More than just a doctor on your phone - Babylo...,https://www.babylonhealth.com/en-us,"Babylon gives you 24/7, all-in-one healthcare ...","['personalized', 'AI', '24/7']"
6,0.6,Sensely: Conversational AI to improve health a...,https://www.sensely.com/,Sensely is an empathy-driven conversational pl...,"['personalized', 'AI', '24/7']"
1,0.2,Youper: Online Therapy With Or Without Insurance,https://www.youper.ai/,"Youper uses Cognitive Behavioral Therapy, the ...",['24/7']
3,0.2,Florence Health,https://florencehealth.com/,Florence is a one-of-a-kind primary care provi...,['24/7']
5,0.2,Health. Powered by Ada.,https://ada.com/,Hi. We're Ada. Our app helps people manage the...,['AI']
